In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/bias-bench
!python -m pip install -e .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/bias-bench
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/bias-bench
  Attempting uninstall: bias-bench
    Found existing installation: bias-bench 0.1.0
    Can't uninstall 'bias-bench'. No files were found to uninstall.
  Running setup.py develop for bias-bench


In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Evaluate original bias

In [ ]:
setting = 'checklist'

In [ ]:
!python experiments/stereoset.py --input_file "./data/stereoset/gender_data_{setting}.json" --model "GPT2LMHeadModel" --model_name_or_path "gpt2" --seed 1 --experiment_name 'implicit_gender_{setting}'

Running StereoSet:
 - input_file: ./data/stereoset/gender_data_checklist.json
 - persistent_dir: /content/drive/MyDrive/bias-bench
 - model: GPT2LMHeadModel
 - model_name_or_path: gpt2
 - batch_size: 1
 - seed: 1
 - experiment_id : stereoset_m-GPT2LMHeadModel_c-gpt2_s-1
 - experiment_name : implicit_gender_checklist
Using mask_token, but it is not set yet.
Evaluating intrasentence task.
100% 523/523 [00:11<00:00, 46.56it/s]


In [ ]:
!python experiments/stereoset_evaluation.py --predictions_file results/stereoset/stereoset_m-GPT2LMHeadModel_c-gpt2_s-1_implicit_gender_checklist.json   --output_file results/stereoset/results_implicit_gender_checklist.json --data_file "./data/stereoset/gender_data_checklist.json"

Evaluating StereoSet files:
 - predictions_file: results/stereoset/stereoset_m-GPT2LMHeadModel_c-gpt2_s-1_implicit_gender_checklist.json
 - predictions_dir: None
 - output_file: results/stereoset/results_implicit_gender_checklist.json
 - data_file: ./data/stereoset/gender_data_checklist.json
intrasentence
	implicit_gender
		Count: 1046.0
		LM Score: 0.0
		SS Score: 69.21606118546845
		ICAT Score: 0.0


Evaluate debiasing performance per prompt

In [ ]:
prompts = [{"gender": "The following text discriminates against people: "},
           {"gender": "The following text contains a bias: "},
           {"gender": "The following text contains biases: "},
           {"gender": "The following text contains a bias agains people: "},
           {"gender": "The following text discriminates against people because of their gender: "},
          {"gender": "The following text implicitly discriminates against people because of their gender: "},
          {"gender": "The following text discriminates implicitly against people because of their gender: "},
          {"gender": "The following text discriminates against the agency of people because of their gender: "},
          {"gender": "The following text limits the agency of people because of their gender: "},
          {"gender": "The following text discriminates against people because of their gender identity: "}]

In [ ]:
for i, prompt in enumerate(prompts):
  print()
  print("NEW PROMPT!!!!!")
  print()

  prompt_str = str(prompt).replace("\'", "\"")
  print(prompt_str)
  print(prompt["gender"])
  prompt_len = len(tokenizer(prompt["gender"])["input_ids"])
  print(prompt_len)
  for n in range(0,5):
      !python experiments/stereoset_debias.py --input_file "./data/stereoset/gender_data_{setting}_{n}.json" --model "SelfDebiasGPT2LMHeadModel" --model_name_or_path "gpt2" --seed 1 --experiment_name 'implicit_gender_{setting}_{n}_prompt{i}' --debiasing_prefixes '{prompt_str}' --bias_position '{prompt_len}'
      !python experiments/stereoset_evaluation.py --predictions_file "results/stereoset/stereoset_m-SelfDebiasGPT2LMHeadModel_c-gpt2_t-gender_s-1_implicit_gender_{setting}_{n}_prompt{i}.json"   --output_file "results/stereoset/results_implicit_gender_{setting}_{n}_prompts{i}.json" --data_file "./data/stereoset/gender_data_{setting}_{n}.json"


NEW PROMPT!!!!!

{"gender": "The following text discriminates against people: "}
The following text discriminates against people: 
9
Running StereoSet:
 - persistent_dir: /content/drive/MyDrive/bias-bench
 - model: SelfDebiasGPT2LMHeadModel
 - model_name_or_path: gpt2
 - bias_direction: None
 - projection_matrix: None
 - load_path: None
 - batch_size: 1
 - bias_type: gender
 - seed: 1
 - experiment_id : stereoset_m-SelfDebiasGPT2LMHeadModel_c-gpt2_t-gender_s-1
 - experiment_name : implicit_gender_checklist_0_prompt0
 - debiasing prefixes : {"gender": "The following text discriminates against people: "}
Using mask_token, but it is not set yet.
Evaluating intrasentence task.
100% 523/523 [00:18<00:00, 27.64it/s]
Evaluating StereoSet files:
 - predictions_file: results/stereoset/stereoset_m-SelfDebiasGPT2LMHeadModel_c-gpt2_t-gender_s-1_implicit_gender_checklist_0_prompt0.json
 - predictions_dir: None
 - output_file: results/stereoset/results_implicit_gender_checklist_0_prompts0.json
 - d

Print and evaluate results per prompt

In [ ]:
import json
import numpy as np

In [ ]:
result_dict = {}
for i, prompt in enumerate(prompts):
  print(prompt)
  prompt_string = str(prompt["gender"]).replace("\'", "\"")
  result_dict[prompt_string] = []
  for n in range(0,5):

      experiment_name = f'implicit_gender_{setting}_{n}'
      file_name = f'results/stereoset/results_{experiment_name}_prompts{i}.json'
      test_name = f'stereoset_m-SelfDebiasGPT2LMHeadModel_c-gpt2_t-gender_s-1_{experiment_name}_prompt{i}'
      with open(file_name, "r") as read_file:
          
          res = json.load(read_file)

      score = res[test_name]['intrasentence']['implicit_gender']['SS Score']
      result_dict[prompt_string].append(score)

{'gender': 'The following text discriminates against people: '}
{'gender': 'The following text contains a bias: '}
{'gender': 'The following text contains biases: '}
{'gender': 'The following text contains a bias agains people: '}
{'gender': 'The following text discriminates against people because of their gender: '}
{'gender': 'The following text implicitly discriminates against people because of their gender: '}
{'gender': 'The following text discriminates implicitly against people because of their gender: '}
{'gender': 'The following text discriminates against the agency of people because of their gender: '}
{'gender': 'The following text limits the agency of people because of their gender: '}
{'gender': 'The following text discriminates against people because of their gender identity: '}


In [ ]:
for prompt in result_dict.keys():
  
  print(f"{prompt} {np.mean(result_dict[prompt])}" )

The following text discriminates against people:  56.86424474187381
The following text contains a bias:  57.36137667304015
The following text contains biases:  57.51434034416826
The following text contains a bias agains people:  56.022944550669216
The following text discriminates against people because of their gender:  56.48183556405354
The following text implicitly discriminates against people because of their gender:  56.367112810707454
The following text discriminates implicitly against people because of their gender:  56.44359464627151
The following text discriminates against the agency of people because of their gender:  56.749521988527725
The following text limits the agency of people because of their gender:  57.09369024856596
The following text discriminates against people because of their gender identity:  55.717017208413004
